In [1]:

%load_ext autoreload
%autoreload 2
from rioplatense_hs.openai import get_completion
from datasets import load_dataset

dataset = load_dataset("piuba-bigdata/contextualized_hate_speech")


In [2]:
import json

raw_dataset = json.load(open("../data/dataset.raw.json"))

type(raw_dataset)

list

In [3]:
dataset["dev"]

Dataset({
    features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
    num_rows: 9106
})

In [4]:
from rioplatense_hs.preprocessing import labels, label_to_text

df_train = dataset["train"].to_pandas()
df_test = dataset["test"].to_pandas()
df_dev = dataset["dev"].to_pandas()
df_train["num_hatred"] = df_train[labels].sum(axis=1)
df_dev["num_hatred"] = df_dev[labels].sum(axis=1)
df_test["num_hatred"] = df_test[labels].sum(axis=1)

df_train["text_label"] = df_train.apply(lambda x: label_to_text(x), axis=1)
df_dev["text_label"] = df_dev.apply(lambda x: label_to_text(x), axis=1)
df_test["text_label"] = df_test.apply(lambda x: label_to_text(x), axis=1)


In [5]:
labels

['WOMEN',
 'LGBTI',
 'RACISM',
 'CLASS',
 'POLITICS',
 'DISABLED',
 'APPEARANCE',
 'CRIMINAL']

In [6]:
df_train[df_train["CRIMINAL"] > 0].sample(2)[["context_tweet", "text"]]

,context_tweet,text
26852,Tres extranjeros incumplían la cuarentena tota...,@usuario Fuera de aca
10525,Video: el momento en que un policía mata a bal...,@usuario Que lindo ver como mueren estas ratas...


In [7]:
import pandas as pd

pd.options.display.max_colwidth = 1000
#df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

In [47]:
from pprint import pprint as pp
from rioplatense_hs.prompting import build_prompt, examples

print(len(examples))
print(build_prompt(contexto="Estados Unidos restringe los vuelos debido al COVID-19", texto="Todo esto es una locura"))

12
Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:

- mujer: Mujeres o movimiento feminista
- lgbti: Género, identidad sexual
- racismo: Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades
- clase: Clase, situación socioeconómica o barrio de residencia
- discapacidad: Discapacidades, problemas de salud mental o de adicción al alcohol u otras drogas
- política: Opinión o ideología política.
- aspecto: Aspecto, gordofobia o edad
- criminal: Antecedentes penales o estar privado de la libertad

Responder una o varias de las características separadas por coma, o bien "nada" si no hay discurso discriminatorio. Pensá p

In [38]:
# sample_df = df_test.sample(100)

# sample_df = df_dev.sample(700, random_state=2022)

sample_df = pd.read_csv("../data/sample_dev_df.csv")

In [39]:
(sample_df["num_hatred"] > 0).mean()

0.16857142857142857

In [40]:
sample_df["prompt"]

0      Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:\n\n- mujer: Mujeres o movimiento feminista\n- lgbti: Género, identidad sexual\n- racismo: Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades\n- clase: Clase, situación socioeconómica o barrio de residencia\n- discapacidad: Discapacidades, problemas de salud mental o de adicción al alcohol u otras drogas\n- política: Opinión o ideología política.\n- aspecto: Aspecto, gordofobia o edad\n- criminal: Antecedentes penales o estar privado de la libertad\n\nResponder una o varias de las características separadas por coma, o bien "nada" si no hay discurso discrimin

In [41]:
sample_df[["context_tweet", "text", "text_label","pred_cot"]].sample(5)

,context_tweet,text,text_label,pred_cot
226,"El video de la pelea de Cinthia Fernández con una vecina: “Me robaron, me pegaron y me tiraron del pelo"" https://t.co/0T1yEKlv4S","@usuario Y fue a pelear ,pobre mina no puede ser más agresiva",nada,"El texto se refiere a Cinthia Fernández como ""agresiva"" debido a su actitud de pelear. La respuesta final es ""mujer""."
675,Romina Malaspina hizo su descargo tras la polémica por su look para el noticiero de Canal 26 https://t.co/hHt7CHzVKN,@usuario La evolución es mostrar las tetas ... que lejos estás de ser periodista GATO,mujer,"El comentario alude a su apariencia (mostrar las tetas) y la insulta llamándola gato. La respuesta final es ""mujer, aspecto"""
379,"Coronavirus en la Argentina: cómo vive Estanislao, el hijo de Alberto Fernández, la cuarentena en Olivos https://t.co/sAuEpeHCui https://t.co/KBKHmq4hIG",@usuario Que interesante!🤣,nada,"El texto no contiene ningún tipo de discurso discriminatorio. La respuesta final es ""nada""."
480,Loly Antoniale mostró su impresionante casa en Miami: “Soy la reina de mi castillo” https://t.co/GuFd1ythLA,@usuario La felicito.\nSabía que está chica haría carrera en el lugar que fuera.,nada,El mensaje no contiene discurso discriminatorio. La respuesta final es 'nada'.
642,"""El país está tomando todas las medidas para afrontar el coronavirus"" https://t.co/F1Zv4dnGCX","@usuario Cientificas medidas como la declaracion jurada, vamos fenomenos",nada,"El texto no contiene discurso discriminatorio. La respuesta final es ""nada""."


## Error analysis

In [42]:
# Check "la respuesta final" is in every pred_cot

sample_df[~sample_df["pred_cot"].str.contains("respuesta final")][["context_tweet", "text", "text_label","pred_cot"]]

,context_tweet,text,text_label,pred_cot
81,Un diputado chileno propuso trasladar a enfermos de coronavirus a la Argentina https://t.co/X7REunq8Mt,"@usuario ""Como ustedes hoy no están tan saturados, podríamos hacer un convenio de aprovechar nosotros, que estamos falto de aquello, y posteriormente, ojalá no ocurra, que pudiese haber un aumento de contagiados, ustedes utilizar la infraestructura nuestra""",nada,"El mensaje no contiene discurso discriminatorio. No realiza ninguna declaración de rechazo, enemistad o aborrecimiento contra un individuo o grupo por poseer una característica protegida."


In [43]:
from rioplatense_hs.preprocessing import text_to_label, labels


pred_labels = [f"PRED_{l}" for l in labels]

for idx, value in sample_df["pred_cot"].items():

    preds = text_to_label(value)

    for k, v in preds.items():
        sample_df.loc[idx, f"PRED_{k}"] = int(v)

In [44]:
from sklearn.metrics import classification_report
gold_labels = sample_df[labels]
preds = sample_df[pred_labels].astype(int)


print(classification_report(gold_labels, preds, target_names=labels))

              precision    recall  f1-score   support

       WOMEN       0.38      0.43      0.40        21
       LGBTI       0.47      0.54      0.50        13
      RACISM       0.59      0.82      0.69        40
       CLASS       0.14      0.50      0.22        10
    POLITICS       0.22      0.50      0.31        22
    DISABLED       0.11      0.20      0.14         5
  APPEARANCE       0.23      0.41      0.30        22
    CRIMINAL       0.33      0.47      0.39        17

   micro avg       0.33      0.55      0.41       150
   macro avg       0.31      0.48      0.37       150
weighted avg       0.37      0.55      0.43       150
 samples avg       0.10      0.10      0.10       150



/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
label_hate = sample_df.num_hatred > 0
pred_hate = sample_df[pred_labels].sum(axis=1) > 0

print(classification_report(label_hate, pred_hate))

              precision    recall  f1-score   support

       False       0.92      0.80      0.86       582
        True       0.40      0.68      0.50       118

    accuracy                           0.78       700
   macro avg       0.66      0.74      0.68       700
weighted avg       0.84      0.78      0.80       700



In [ ]:
df_train

In [37]:
sample_df[pred_hate & ~label_hate][["context_tweet", "text", "text_label", "pred_cot", ]].sample(10)

[autoreload of rioplatense_hs.prompting failed: Traceback (most recent call last):
  File "/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/jmperez/.pyenv/versions/3.10.7/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/jmperez/projects/rioplatense_hate_sp

,context_tweet,text,text_label,pred_cot
29,"Elisa Carrió: ""No tengan miedo, el problema no es el coronavirus sino Putin"" https://t.co/PNCHuFxdnM",@usuario Pongan en cuarentena a está mujer...lo tiene lo tiene,nada,"El texto alude a que Elisa Carrió tiene problemas de salud mental y sugiere que debería estar en cuarentena, lo cual hace referencia a su discapacidad. La respuesta final es ""discapacidad"""
32,"Luis Brandoni cruzó a Ricardo Alfonsín por sus críticas a la marcha: ""No está en el centro de sus convicciones, de lo que ha mamado desde que nació"" https://t.co/RuqteYDjek",@usuario Que convicciones querés que tenga el excremento de @usuario si manotea un millón de pesos por mes del estado. A quien carajo le puede importar la moral que pueda tener un hdp así.,nada,"El texto se refiere al excremento como alguien que recibe un millón de pesos del estado, lo cual puede considerarse una crítica hacia su situación socioeconómica. La respuesta final es ""clase""."
26,"Es norteamericano, sobrevivió al 11-S y se mudó a Argentina: “Es uno de los mejores países del mundo” | Por Florencia Illbele\nhttps://t.co/Lsiw2RrMnR","@usuario Obvio míster, lástima el #gobiernodemierda que tenemos y todos sus séquitos planeros. Andate antes q te apuñalen en la calle y la policía no te pueda defender.",nada,"El texto insulta al gobierno y a quienes reciben planes sociales, aludiéndolos como ""planeros"". La respuesta final es ""clase"""
31,"Grabois calificó al campo como ""parásitos que viven de la renta extraordinaria de la tierra que es de todos"" https://t.co/c8XQr9c1aA","@usuario LA TIERRA ES PARA EL QUE LA COMPRO CON ESFUERZO Y SUDOR DE SANGRE, PARA QUE VENGAS VOS IMBECIL A QUERER DECIR QUE ES DE TODOS. NO SABES DIFERENCIAR UNA VACA DE UN CHIMANGO.",nada,"El texto contiene insultos (""imbécil"") y una expresión de rechazo hacia el grupo al que pertenece (Grabois y su opinión sobre el campo), pero no hace referencia directa a las características protegidas. La respuesta final es ""política""."
65,"China celebró el final de su batalla contra el coronavirus: ""Hemos derrotado al actual brote"" https://t.co/b6QmYSjawg",@usuario El famoso cuento chino...,nada,"El texto utiliza la expresión ""cuento chino"" de manera peyorativa, lo cual puede ser considerado como un estereotipo o prejuicio hacia los chinos. La respuesta final es ""racismo"""
62,"Guillermo Moreno: ""Es una barbaridad cerrar las villas, el peronismo es amor, ¿qué hubiese dicho Evita?"" https://t.co/FWMNdyO2Qk",@usuario Amor......... a la pobreza,nada,"El texto se refiere al peronismo como amor a la pobreza. La respuesta final es ""política""."
85,🔴 AHORA | El gobierno porteño adelantó que la nueva cuarentena tendrá una fecha de finalización concreta https://t.co/Mu5y2vf9bw https://t.co/N3Z61jmfAJ,@usuario No seas trolo infogarcha. Que es esa foto de perfil?,nada,"El texto utiliza un insulto homofóbico (""trolo"") y una expresión despectiva (""infogarcha"") para referirse a la persona a la que está respondiendo. Además, se burla de su foto de perfil. La respuesta final es ""lgtbi, aspecto""."
98,"La frase homofóbica de Jair Bolsonaro al hablar del COVID-19 en Brasil: “Todos nos vamos a morir un día, dejen de ser un país de maricas” https://t.co/QDn9vMUYHh",@usuario El Sr. Presidente de Brasil tiene toda la razón !!!,nada,"El texto apoya y respalda el discurso homofóbico de Jair Bolsonaro. La respuesta final es ""lgbti, política"""
41,La desacertada declaración de Bernie Ecclestone: “En muchos casos las personas negras son más racistas que las blancas” https://t.co/30IR1woaCw,@usuario Tiene razón. No es políticamente correcto decirlo. Pero es cierto,nada,"El texto respalda una declaración discriminatoria al afirmar que las personas negras son más racistas que las blancas. La respuesta final es ""racismo"""
18,"Coronavirus: con un piquete fronterizo, bolivianos piden controles sanitarios para los argentinos que entran a ese país https://t.co/EwPYLuDXlA","@usuario Acá somos tan bolu

In [36]:
df_train[df_train["num_hatred"] == 0][["context_tweet", "text", "text_label"]].sample(10)

,context_tweet,text,text_label
22340,La historia de la modelo colombiana trans que besa la panza de su esposo embarazado de ocho meses https://t.co/9eSCC1eOHj,@usuario Imaginate llorar por eso kjjj,nada
13928,Flor de equipo: así será el nuevo magazine de Telefe https://t.co/WSVHSV5ajd https://t.co/5AEkCzX8T9,@usuario Raro que esta señora no fue al canal estatal !!! Trabaja para una corporación???? O no es corpo?,nada
14100,"Barack Obama: ""La guerra a veces es necesaria "" https://t.co/4PgQhChTnY https://t.co/CxKBWKTCfd",@usuario Buah,nada
32731,"Joe Biden nominó a Rachel Levine, una mujer transgénero, para que sea su subsecretaria de Salud https://t.co/xqIA6f1XFK",@usuario Excelente!,nada
35472,Revocaron la prisión domiciliaria del violador que acosaba a su vecina de 12 años en Tucumán https://t.co/cgtgKeDthM,"@usuario Un rayo la justicia , unos teflejos!!",nada
13893,Una dirigente gremial docente afirmó que “el nivel de circulación de coronavirus dentro de una escuela es tan alto que podría compararse con el de una fiesta clandestina” https://t.co/4Huf0N317Q,@usuario Andá a laburar lcdtm,nada
27345,Nicolás Maduro ahora le pide auxilio económico al FMI para combatir al coronavirus https://t.co/NwNAqLZIoe,"@usuario Que raro no le pidió ayuda a sus amigos dictadores de Corea del Norte, Cuba, China o Rusia",nada
10443,Alarma entre los científicos en Bolivia por la aparición de un nuevo virus con una letalidad similar al ébola https://t.co/qHLVgRSMRe,@usuario La ptm 🤦‍♂️,nada
22615,"El Gobierno advirtió que habrá un ""pico de delitos contra la propiedad” cuando termine la cuarentena https://t.co/s4LbjLgJ0N",@usuario Ya hay,nada
25955,Coronavirus: el Papa envió ayuda a transexuales que se quedaron sin clientes https://t.co/eqVgFsDse3 https://t.co/4VeltGuIbB,@usuario Es joda no?,nada


In [177]:
# LGBTI errors

label = "CLASS"
sample_df.loc[sample_df[label] != sample_df[f"PRED_{label}"]][["context_tweet", "text", "text_label", "pred_cot"]].sample(5)

,context_tweet,text,text_label,pred_cot
690,"En promedio, los beneficiarios de planes sociales cobrarán $40.000 en diciembre | Por Pablo Wende https://t.co/OaSD2ZmmHy",@usuario @usuario Les seguimos pagando el crédito de ANSES con el q se compraron el 50’ \nArroyo mismo lo dijo cuando empezó la cuareterna: -muchos tienen créditos y les llega descontado del plan”-\nPagamos sus vicios y sus “gustitos “,nada,"El texto se refiere a los beneficiarios de planes sociales como personas que gastan el dinero en vicios y gustitos. La respuesta final es ""clase"""
294,"Juan Grabois, sobre la post pandemia: “Si no hay una intervención política, va a haber un fuerte aumento de la criminalidad y un proceso de toma de tierras” https://t.co/h5wlZGPRQd",@usuario Dnd vive Grabois ?? En algún asentamiento ó barrio privado ??,nada,"El texto cuestiona el lugar de residencia de Juan Grabois, sugiriendo la posibilidad de que viva en un asentamiento o barrio privado. La respuesta final es ""clase"""
682,"Juan Grabois: ""Hay una oleada de toma de tierras y se va a profundizar "" https://t.co/2B1JL2BkHh https://t.co/jJXbT0X564","@usuario Espero que la gente, dueña,.los saque a patadas en el culo. Quien se creen que son. Tomar tierra, trabaja y pedí un crédito. Por Asís e hacen las cosas, así hacen el resto de los argentinos que trabaja.","clase, criminal",El texto no contiene discurso discriminatorio. La respuesta final es 'nada'
149,El Polo Obrero realiza un corte en la Panamericana en contra de la flexibilización de la cuarentena y en reclamo de aumentos a los planes sociales https://t.co/fHOeCKJdY6,"@usuario A ver si entendí, cortan para q nadie vaya a laburar y q leaumente la limosna ??!!!",nada,"El texto critica a los manifestantes por cortar la Panamericana y reclamar aumentos a los planes sociales, llamándolos ""limosna"". La respuesta final es ""clase"""
93,Pablo Moyano amenaza con un paro de recolectores en la Ciudad de Buenos Aires https://t.co/tPArqK6otB https://t.co/8F6nHRi0Jr,@usuario CUANTA PLATA TIENEN ENTRE ESTOS TRES TIPOS. SON MULTIMILLONARIOS.,nada,"El texto se refiere a la cantidad de dinero que tienen entre los tres mencionados. La respuesta final es ""clase"""


In [117]:
df_train[(df_train["POLITICS"] > 0) & (df_train["num_hatred"] > 1)][["context_tweet", "text", "text_label"]].sample(5)

,context_tweet,text,text_label
32287,"Cecilia Moreau: ""No se puede permitir que la minoría le imponga a la mayoría qué temas discutir"" https://t.co/DIOKystxnf",@usuario COCHINA MENTIROSA!! CÓMPLICE canalla de la mafia KAKA!!,"política, aspecto"
26017,Coronavirus en Argentina: el Gobierno convocará a miles de piqueteros para convencer a la gente que se vacune https://t.co/XofDz33SV1,@usuario Los quieren concencer a los palazos? Vacunena los piqueteros y pkanwros primero. @usuario @usuario @usuario @usuario @usuario,"clase, política"
7622,"""Día menstrual para estudiantes "": el proyecto de ley de Carmela Moreau https://t.co/OfxmKpX7bT https://t.co/YxTJbSrQ3k","@usuario Gobierno nacional popular y generador de excusas para no trabajar, y digo gobierno x que esta inepta, incapaz, indigna y descerebrada de Carmela tiene el sello kk ya incorporado en el adn #GobiernoDeInutiles #gobiernodemierda #GobiernoDeMierda","mujer, política"
28206,Santa Fe: rompen más silobolsas y crece la preocupación en el campo https://t.co/Pb1gaBqyXU https://t.co/cPlGjy3V6T,"@usuario hay que hacer guardia, meter bala y enterrar a los soretes para abonar la tierra","política, criminal"
28983,'País de maricas': ¿Por qué Jair Bolsonaro vuelve a atacar a los homosexuales en Brasil? https://t.co/pVdFjiztkm,"@usuario El discurso de bolsonaro donde dice ""maricas"" es brillante, sólo eso basta para meresca ser el presidente de Sudamérica. Clarín es una cueva de zurdos y progresistas que se pintan los labios punto. Clarín suavecito, te pasas de marica!","lgbti, política"
